# Brief Métier : Exploitation des Données Scrappées avec Pandas

## Contexte Professionnel

Après la phase de **scraping** réalisée sur des sites concurrents de Castorama, vous avez obtenu deux fichiers CSV :

- categories.csv : contenant les informations relatives aux catégories et sous-catégories.
- products.csv : contenant les informations relatives aux produits (nom, prix, disponibilité, promotions, etc.).

En tant que **Data Analyst / Data Engineer**, votre rôle est désormais de **nettoyer**, **préparer** et **analyser** ces données afin d’en extraire des **informations pertinentes**. Ces informations permettront à Castorama de mieux comprendre l’état du marché, de mettre en place une stratégie tarifaire compétitive et d'anticiper les tendances.

## Installations

In [ ]:
!pip install --upgrade pip
!pip install ipykernel
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn

## Importations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Chargement des Données

In [2]:
# Load category data

category_data = pd.read_csv("castorama_categories.csv")

# Load products data

product_data = pd.read_csv("castorama_products.csv")

In [ ]:
# Confirm category data loaded correctly

category_data

In [ ]:
# Confirm product data loaded correctly

product_data

## Exploration

### Aperçu des données

In [ ]:
# Get information about category_data

category_data.info()

In [ ]:
# Get information about product_data

product_data.info()

In [ ]:
# View category_data summary statistics 

category_data.describe()

In [ ]:
# View product_data summary statistics 

product_data.describe()

In [ ]:
# View first 5 data in category_data

category_data.head()

In [ ]:
# View first 5 data in product_data

product_data.head()

## Nettoyage et Préparation des Données

### Category_data.csv

In [3]:
# Duplicate raw data

cdf = category_data.copy()

In [ ]:
# Check for missing data

cdf.isna().sum()

In [4]:
# View duplicated categories 

duplicates = cdf[cdf["category"].duplicated(keep=False)]

In [ ]:
# Sort duplicates to understand patterns

duplicates.sort_values(by=['is_page_list','category', 'url'], ascending=False)

In [ ]:
# Sort data by the specified columns

cdf.sort_values(by=['is_page_list','category', 'url'], ascending=False)

In [ ]:
# Drop duplicates (keep only first occurrence)

cdf.drop_duplicates(subset=["category"], inplace=True, keep='first')

# View data
cdf.describe()

In [ ]:
# View duplicated urls

duplicated_urls = cdf[cdf["url"].duplicated(keep=False)]

# Sort by url
duplicates_sorted = duplicated_urls.sort_values(by="is_page_list", ascending=False)

# View data
duplicates_sorted

In [ ]:
# Sort data by "is_page_list"

cdf.sort_values(by='is_page_list', ascending=False)

In [ ]:
# Drop duplicate urls (Observation: Double is_page_list created for SEO and Diacritics)

cdf.drop_duplicates(subset=["url"], inplace=True, keep='first')

# Summarize data
cdf.describe()

- Manipulation de chaînes :

In [23]:
# Remove trailing spaces and characters in category name

cdf["category"] = cdf["category"].str.strip()

In [24]:
# Convert category names to lowercase

cdf['category'] = cdf['category'].str.lower()

In [25]:
# Handling encoding issues (trailing underscores, Prefix 0s, multiple underscores) in specified columns
# Replace spaces, commas, apostrophes with underscore

import re

def clean_text(input_str):
    if pd.isnull(input_str):  # Handle NaN values
        return input_str
    input_str = str(input_str)  # Ensure the input is a string
    input_str = re.sub(r'[\u002D\u2010\u2011\u2012\u2013\u2014\u2212]', '_', input_str) # Replace all hyphen types
    input_str = re.sub(r'\s+', '_', input_str.strip())  # Replace all whitespace with underscores
    input_str = input_str.replace(',', '_')  # Replace commas with underscores
    input_str = input_str.replace("'", '_')  # Replace apostrophes with underscores
    input_str = input_str.replace('\xa0', '_')  # Replace non-breaking spaces
    input_str = re.sub(r'_+', '_', input_str)  # Remove multiple underscores
    input_str = re.sub(r'^_|_$', '', input_str)  # Remove leading or trailing underscores
    input_str = re.sub(r'^0+', '', input_str)  # Remove leading zeros
    return input_str

cdf['category'] = cdf['category'].map(clean_text)

In [26]:
# Remove accents

replacements = {"à": "a", "á": "a", "â": "a", "ä": "a", "ç" : "c",
                "é": "e", "è": "e", "ê": "e", "ë": "e", "É":"E", "È":"E",
                "î": "i", "ï":"i", "ì": "i", "í": "i",
                "ö": "o", "ô": "o", "ò": "o", "ó": "o",
                "ü": "u", "û": "u", "ù": "u", "ú": "u"}

def replace_accents(input_str, replacement):
    for old, new in replacement.items():
        input_str = input_str.replace(old, new)
    return input_str

cdf["category"] = cdf["category"].apply(lambda x: replace_accents(str(x), replacements))

In [ ]:
# Review for errors or duplicates

cdf.sort_values(by='category')

In [ ]:
# Verify again if duplicates persists

c_duplicates = cdf[cdf['category'].duplicated(keep=False)]

c_duplicates.sort_values(by='category')

In [29]:
# Sort by is_page_list

cdf_sorted = cdf.sort_values(by="is_page_list", ascending=False)

In [32]:
# Drop newly found duplicates (SEO / Diacritics related, keep only "True" is_page_lists)

cdf_no_duplicates = cdf_sorted.drop_duplicates(subset=['category'], keep='first')

In [33]:
# Sort by original index

cdf_no_duplicates = cdf_no_duplicates.sort_index()

In [34]:
# Duplicate original index

cdf_no_duplicates["original_index"] = cdf_no_duplicates.index

In [35]:
# Reset index

categories_cleaned = cdf_no_duplicates.reset_index(drop=True)

In [37]:
# Exclude original index from final copy

categories_cleaned_final = categories_cleaned[['category', 'is_page_list', 'url']]

In [39]:
# Export cleaned data

categories_cleaned_final.to_csv("categories_cleaned_final.csv", index=False)

### Product_data.csv

In [40]:
# Duplicate raw product data

pdf = product_data.copy()

In [ ]:
# Get info about products data

pdf.info()

In [ ]:
# Get summary statistics/info

pdf.describe()

In [ ]:
# Check for missing values (general)

pdf.isna()

In [ ]:
# Check for missing values in unique_id column

pdf["unique_id"].isna().value_counts()

In [ ]:
# Check for missing values (category column)

pdf["category"].isna().value_counts()

In [ ]:
# Check for missing values (subcategory column)

pdf["subcategory"].isna().value_counts()

In [ ]:
# Check for missing values (subsubcategory column)

pdf["subsubcategory"].isna().value_counts()

In [ ]:
# Check for missing values (subsubsubcategory column)

pdf["subsubsubcategory"].isna().value_counts()

In [ ]:
# Check for missing values (price column)

pdf["price"].isna().value_counts()

In [ ]:
# Check for missing values (title column) 

pdf["title"].isna().value_counts()

In [ ]:
# Check for missing values (url column)

pdf["url"].isna().value_counts()

In [45]:
pdf.describe()

,unique_id,category,subcategory,subsubcategory,subsubsubcategory,title,price,url
count,28347,28347,28347,28347,13428,28347,28347,28347
unique,28347,16,52,150,75,27665,5740,28347
top,7610583118042_CAFR.prd,salle-de-bains-et-wc,decoration-textile,coussin-plaid-et-pouf,tapis,Tapis de bain uni en polyester 50x80cm,"19,90",https://www.castorama.fr/salle-de-bains-et-wc/accessoires-de-salle-de-bains/porte-serviettes-et-patere/cat_id_1502.cat/mkp/spirella-porte-serviette-2-barres-lemon-44-cm-blanc/7610583118042_CAFR.prd
freq,1,8456,8096,3550,1589,15,325,1


In [ ]:
# pd.reset_option('display.max_rows', None)

duplicates_in_pdf = pdf[pdf.duplicated(subset="title", keep=False)]

duplicates_in_pdf

- Manipulation de chaînes :

In [ ]:
# Replace NaNs

pdf['subsubsubcategory'] = pdf['subsubsubcategory'].fillna("Not_available")

In [51]:
# Convert specified columns to lowercase 

columns_to_lowercase = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

pdf[columns_to_lowercase] = pdf[columns_to_lowercase].apply(lambda x: x.str.lower())

In [52]:
# Strip values in the specified columns 

columns_to_strip = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

pdf[columns_to_strip] = pdf[columns_to_strip].apply(lambda x: x.str.strip())

In [53]:
# Handling encoding issues (trailing underscores, Prefix 0s, multiple underscores) in specified columns
# Replace spaces, commas, apostrophes with underscore

import re

columns_to_replace = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

def clean_text(input_str):
    if pd.isnull(input_str):  # Handle NaN values
        return input_str
    input_str = str(input_str)  # Ensure the input is a string
    input_str = re.sub(r'[\u002D\u2010\u2011\u2012\u2013\u2014\u2212]', '_', input_str) # Replace all hyphen types
    input_str = re.sub(r'\s+', '_', input_str.strip())  # Replace all whitespace with underscores
    input_str = input_str.replace(',', '_')  # Replace commas with underscores
    input_str = input_str.replace("'", '_')  # Replace apostrophes with underscores
    input_str = re.sub(r'_+', '_', input_str)  # Remove multiple underscores
    input_str = re.sub(r'^_|_$', '', input_str)  # Remove leading or trailing underscores
    input_str = re.sub(r'^0+', '', input_str)  # Remove leading zeros
    return input_str

pdf[columns_to_replace] = pdf[columns_to_replace].map(clean_text)

In [54]:
# Replace accented letters in the specified columns' texts

import unicodedata

columns_to_replace = ['category', 'subcategory', 'subsubcategory', 'subsubsubcategory', 'title']

def robust_remove_accents(input_str):
    # Normalize to decomposed form
    normalized = unicodedata.normalize('NFD', input_str)
    # Remove combining characters (accents)
    without_accents = ''.join(c for c in normalized if unicodedata.category(c) != 'Mn')

    # Explicitly replace problematic characters (if any remains)
    replacements = {"à": "a", "á": "a", "â": "a", "ä": "a", "ç": "c", "ć": "c",
                "é": "e", "è": "e", "ê": "e", "ë": "e", "É":"E", "È":"E",
                "î": "i", "ï":"i", "ì": "i", "í": "i",
                "ö": "o", "ô": "o", "ò": "o", "ó": "o",
                "ü": "u", "û": "u", "ù": "u", "ú": "u"}

    for accented_char, replacement in replacements.items():
        without_accents = without_accents.replace(accented_char, replacement)
    
    # Handle lingering issues and strip
    return without_accents.replace('\xa0', ' ').strip()

pdf[columns_to_replace] = pdf[columns_to_replace].map(
    lambda x: robust_remove_accents(str(x)) if isinstance(x, str) else x)

In [ ]:
# Convert price column to float type

pdf['price'] = pdf['price'].apply(lambda x: x.replace(",", "."))
pdf['price'] = pdf['price'].apply(lambda x: x.replace(" ", ""))

pdf['price'] = pd.to_numeric(pdf['price'], errors='coerce')

In [ ]:
# Display count of prices with NaN

pdf['price'].isna().value_counts()

In [ ]:
# Show rows with NaNs to understand the problem

pdf_nas = pdf[pdf.isna().any(axis=1)]

pdf_nas

In [62]:
# Replace NaNs with None

pdf['price'] = pdf['price'].replace({pd.NA: None, np.nan: None})

# Drop rows with NaN
pdf = pdf.dropna(subset=['price'])

In [67]:
# Store copy of original index

pdf['original_index'] = pdf.index

In [70]:
# Reset index

products_cleaned = pdf.reset_index(drop=True)

In [83]:
products_cleaned['price'] = pd.to_numeric(products_cleaned['price'], errors='coerce')

products_cleaned_final = products_cleaned[['unique_id','category','subcategory', 'subsubcategory', 'subsubsubcategory', 'title', 'price', 'url']]

In [85]:
products_cleaned_final.to_csv("products_cleaned_final.csv", index=False)